This notebook includes the main procedures of cross-match between ICRF3 and Gaia-DR2 catalogs.

In [1]:
import time
import numpy as np
from astropy.table import Table, join, Column
from astropy import units as u

# My progs
from my_progs.catalog.read_icrf import read_icrf3
from my_progs.catalog.read_gaia import read_dr2_iers
from my_progs.catalog.pos_diff import radio_cat_diff_calc

Get the Gaia DR2 data.

In [2]:
gdr2 = read_dr2_iers()

gdr2_sou = Table(gdr2)
gdr2_sou.keep_columns(["iers_name"])

Crossmatch with ICRF3 S/X catalog.

In [3]:
icrf3sx = read_icrf3(wv="sx")
icrf3k = read_icrf3(wv="k")
icrf3xka = read_icrf3(wv="xka")

icrf3sx_sou = Table(icrf3sx)
icrf3k_sou = Table(icrf3k)
icrf3xka_sou = Table(icrf3xka)

icrf3sx_sou.keep_columns(["iers_name"])
icrf3k_sou.keep_columns(["iers_name"])
icrf3xka_sou.keep_columns(["iers_name"])

In [4]:
temp1 = join(icrf3sx_sou, gdr2_sou, keys="iers_name")

temp2 = join(icrf3k_sou, icrf3xka_sou, keys="iers_name")

com_list = join(temp1, temp2, keys="iers_name")

In [5]:
com_list.meta["comments"] = [
    "List of {} common sources between ICRF3 (all 3 bands) and Gaia-CRF2".
    format(len(com_list)), "Created at {}".format(time.asctime())
]
com_list.write("../data/com-sou-list-DR2.txt", format="ascii", overwrite=True)

In [6]:
pos_oft = radio_cat_diff_calc(icrf3sx, gdr2, sou_name="iers_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra', 'ra_err_2', 'dec', 'dec_err_2'])

pos_oft1 = radio_cat_diff_calc(gdr2, icrf3sx, sou_name="iers_name")
pos_oft1.remove_columns(['ra_err_1', 'dec_err_1', 'ra', 'ra_err_2', 'dec', 'dec_err_2'])

In [7]:
gdr2_icrf3sx = join(gdr2, icrf3sx, keys='iers_name', table_names=['gdr2', 'icrf3sx'])
gdr2_icrf3sx_oft = join(gdr2_icrf3sx, pos_oft, keys='iers_name')

gdr2_icrf3sx_oft.rename_columns(['ra_gdr2', 'dec_gdr2'],
                                 ['ra', 'dec'])

In [8]:
icrf3sx_gdr2_oft = join(gdr2_icrf3sx, pos_oft1, keys='iers_name')
icrf3sx_gdr2_oft.rename_columns(['ra_icrf3sx', 'dec_icrf3sx'],
                                 ['ra', 'dec'])

In [9]:
gdr2_icrf3sx_oft.write('../data/icrf3sx_gdr2.fits', overwrite=True)
icrf3sx_gdr2_oft.write('../data/gdr2_icrf3sx.fits', overwrite=True)

In [10]:
icrf3k = read_icrf3(wv="k")

pos_oft = radio_cat_diff_calc(icrf3k, gdr2, sou_name="iers_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
                        'ra_err_2', 'dec', 'dec_err_2'])

gdr2_icrf3k = join(gdr2, icrf3k, keys='iers_name',
                    table_names=['gdr2', 'icrf3k'])
gdr2_icrf3k_oft = join(gdr2_icrf3k, pos_oft, keys='iers_name')
gdr2_icrf3k_oft.rename_columns(['ra_gdr2', 'dec_gdr2'],
                                ['ra', 'dec'])

gdr2_icrf3k_oft.write('../data/icrf3k_gdr2.fits', overwrite=True)

In [11]:
icrf3xka = read_icrf3(wv="xka")

pos_oft = radio_cat_diff_calc(icrf3xka, gdr2, sou_name="iers_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
                        'ra_err_2', 'dec', 'dec_err_2'])

gdr2_icrf3xka = join(gdr2, icrf3xka, keys='iers_name',
                      table_names=['gdr2', 'icrf3xka'])
gdr2_icrf3xka_oft = join(gdr2_icrf3xka, pos_oft, keys='iers_name')

gdr2_icrf3xka_oft.rename_columns(['ra_gdr2', 'dec_gdr2'],
                                  ['ra', 'dec'])

gdr2_icrf3xka_oft.write('../data/icrf3xka_gdr2.fits', overwrite=True)